# Overview
This is **not** a strategy-edge study. This is a **constraint-satisfaction + worst-case portfolio construction** problem. We are finding scaling factor $s_1$, $s_2$ such that the combined equity process respects **daily** and **total** drawdown constraints with high confidence.

In [ ]:
# Imports
import pandas as pd
import numpy as np

### 1. Loading and normalizing the data
df_z6 = pd.read_csv('C:\\Users\\blazo\\Documents\\Misc\\QJ\\quant-journey\\research\\ftmo\\data\\Z6+_pnl.csv')
df_z7 = pd.read_csv('C:\\Users\\blazo\\Documents\\Misc\\QJ\\quant-journey\\research\\ftmo\\data\\Z7_pnl.csv')

# Convert Date to datetime
df_z6['Date'] = pd.to_datetime(df_z6['Date'])
df_z7['Date'] = pd.to_datetime(df_z7['Date'])

# Convert PnL to Equity (starting equity = 10000)
df_z6['Equity'] = 10000 + df_z6['PnL']
df_z7['Equity'] = 10000 + df_z7['PnL']

# Convert Equity to Returns
df_z6['Returns'] = df_z6['Equity'].pct_change()
df_z7['Returns'] = df_z7['Equity'].pct_change()

In [42]:
### 2. Individual strategy risk diagnostics

# Helper functions for cleaner code
def calculate_drawdown_metrics(df):
    """Calculate drawdown metrics for a strategy dataframe."""
    df['Peak'] = df['Equity'].cummax()
    df['Drawdown'] = ((df['Equity'] - df['Peak']) / df['Peak']) * 100
    
    max_dd = df['Drawdown'].min()
    max_dd_idx = df['Drawdown'].idxmin()
    peak_idx = df.loc[:max_dd_idx, 'Peak'].idxmax()
    
    start_date = df.loc[peak_idx, 'Date']
    trough_date = df.loc[max_dd_idx, 'Date']
    
    # Find recovery date
    recovery_idx = df[(df.index > max_dd_idx) & 
                      (df['Equity'] >= df.loc[peak_idx, 'Peak'])].index
    end_date = df.loc[recovery_idx[0], 'Date'] if len(recovery_idx) > 0 else df.iloc[-1]['Date']
    
    return {
        'max_dd': max_dd,
        'start_date': start_date,
        'trough_date': trough_date,
        'end_date': end_date,
        'days_to_trough': (trough_date - start_date).days,
        'days_total': (end_date - start_date).days
    }

def calculate_daily_loss_metrics(df):
    """Calculate maximum daily loss metrics for a strategy dataframe."""
    worst_day_idx = df['Returns'].idxmin()
    worst_day_return = df['Returns'].min() * 100
    
    return {
        'worst_return': worst_day_return,
        'date': df.loc[worst_day_idx, 'Date'],
        'equity_before': df.loc[worst_day_idx - 1, 'Equity'],
        'equity_loss': df.loc[worst_day_idx, 'Equity'] - df.loc[worst_day_idx - 1, 'Equity']
    }

def calculate_volatility_metrics(df):
    """Calculate volatility metrics for a strategy dataframe."""
    std_dev = df['Returns'].std() * 100
    negative_returns = df[df['Returns'] < 0]['Returns']
    downside_dev = negative_returns.std() * 100
    
    return {
        'std_dev': std_dev,
        'downside_dev': downside_dev,
        'negative_days': len(negative_returns)
    }

def analyze_losing_streaks(df):
    """Analyze consecutive losing days (negative returns)."""
    losing_days = df['Returns'] < 0
    streaks = []
    current_streak = 0
    
    for is_losing in losing_days:
        if is_losing:
            current_streak += 1
        else:
            if current_streak > 0:
                streaks.append(current_streak)
            current_streak = 0
    
    if current_streak > 0:
        streaks.append(current_streak)
    
    if len(streaks) > 0:
        return {
            'total_streaks': len(streaks),
            'mean': np.mean(streaks),
            'median': np.median(streaks),
            'percentile_90': np.percentile(streaks, 90),
            'max': max(streaks),
            'streaks': streaks
        }
    else:
        return {
            'total_streaks': 0,
            'mean': 0,
            'median': 0,
            'percentile_90': 0,
            'max': 0,
            'streaks': []
        }

def print_section_header(title):
    """Print a formatted section header."""
    print("\n" + "=" * 70)
    print(f"  {title}")
    print("=" * 70)

def print_strategy_metrics(strategy_name, metrics_dict):
    """Print metrics in a clean, formatted way."""
    print(f"\n{strategy_name}:")
    print("-" * 70)
    for key, value in metrics_dict.items():
        print(f"  {key:.<50} {value}")

# ============================================================================
# 2.1. Maximum Drawdown Analysis
# ============================================================================
print_section_header("2.1. MAXIMUM DRAWDOWN ANALYSIS")

z6_dd = calculate_drawdown_metrics(df_z6)
z7_dd = calculate_drawdown_metrics(df_z7)

print_strategy_metrics("Z6", {
    "Maximum Drawdown": f"{z6_dd['max_dd']:.2f}%",
    "Peak Date": z6_dd['start_date'].strftime('%Y-%m-%d'),
    "Trough Date": z6_dd['trough_date'].strftime('%Y-%m-%d'),
    "Recovery Date": z6_dd['end_date'].strftime('%Y-%m-%d'),
    "Days to Trough": z6_dd['days_to_trough'],
    "Total Duration (days)": z6_dd['days_total']
})

print_strategy_metrics("Z7", {
    "Maximum Drawdown": f"{z7_dd['max_dd']:.2f}%",
    "Peak Date": z7_dd['start_date'].strftime('%Y-%m-%d'),
    "Trough Date": z7_dd['trough_date'].strftime('%Y-%m-%d'),
    "Recovery Date": z7_dd['end_date'].strftime('%Y-%m-%d'),
    "Days to Trough": z7_dd['days_to_trough'],
    "Total Duration (days)": z7_dd['days_total']
})

# ============================================================================
# 2.2. Maximum Daily Loss Analysis
# ============================================================================
print_section_header("2.2. MAXIMUM DAILY LOSS ANALYSIS")

z6_loss = calculate_daily_loss_metrics(df_z6)
z7_loss = calculate_daily_loss_metrics(df_z7)

print_strategy_metrics("Z6", {
    "Worst Single-Day Return": f"{z6_loss['worst_return']:.2f}%",
    "Date": z6_loss['date'].strftime('%Y-%m-%d'),
    "Equity Before": f"${z6_loss['equity_before']:,.2f}",
    "Equity Loss": f"${z6_loss['equity_loss']:,.2f}"
})

print_strategy_metrics("Z7", {
    "Worst Single-Day Return": f"{z7_loss['worst_return']:.2f}%",
    "Date": z7_loss['date'].strftime('%Y-%m-%d'),
    "Equity Before": f"${z7_loss['equity_before']:,.2f}",
    "Equity Loss": f"${z7_loss['equity_loss']:,.2f}"
})

# ============================================================================
# 2.3. Volatility Analysis
# ============================================================================
print_section_header("2.3. VOLATILITY ANALYSIS")

z6_vol = calculate_volatility_metrics(df_z6)
z7_vol = calculate_volatility_metrics(df_z7)

print_strategy_metrics("Z6", {
    "Standard Deviation (Daily Returns)": f"{z6_vol['std_dev']:.2f}%",
    "Downside Deviation": f"{z6_vol['downside_dev']:.2f}%",
    "Number of Negative Days": z6_vol['negative_days']
})

print_strategy_metrics("Z7", {
    "Standard Deviation (Daily Returns)": f"{z7_vol['std_dev']:.2f}%",
    "Downside Deviation": f"{z7_vol['downside_dev']:.2f}%",
    "Number of Negative Days": z7_vol['negative_days']
})

# ============================================================================
# 2.4. Losing Streak Analysis
# ============================================================================
print_section_header("2.4. LOSING STREAK ANALYSIS")

z6_streaks = analyze_losing_streaks(df_z6)
z7_streaks = analyze_losing_streaks(df_z7)

# Print Z6 streak metrics
print_strategy_metrics("Z6", {
    "Total Number of Losing Streaks": z6_streaks['total_streaks'],
    "Mean Losing Streak": f"{z6_streaks['mean']:.2f} days",
    "Median Losing Streak": f"{z6_streaks['median']:.0f} days",
    "90th Percentile": f"{z6_streaks['percentile_90']:.0f} days",
    "Maximum Streak": f"{z6_streaks['max']} days"
})

# Print Z6 distribution
if z6_streaks['streaks']:
    print("\n  Streak Distribution (Z6):")
    unique_lengths = sorted(set(z6_streaks['streaks']))
    for length in unique_lengths[:10]:
        count = z6_streaks['streaks'].count(length)
        percentage = (count / z6_streaks['total_streaks']) * 100
        print(f"    {length} day(s): {count:>3} occurrences ({percentage:>5.1f}%)")
    if len(unique_lengths) > 10:
        print(f"    ... and {len(unique_lengths) - 10} more unique streak lengths")

# Print Z7 streak metrics
print_strategy_metrics("Z7", {
    "Total Number of Losing Streaks": z7_streaks['total_streaks'],
    "Mean Losing Streak": f"{z7_streaks['mean']:.2f} days",
    "Median Losing Streak": f"{z7_streaks['median']:.0f} days",
    "90th Percentile": f"{z7_streaks['percentile_90']:.0f} days",
    "Maximum Streak": f"{z7_streaks['max']} days"
})

# Print Z7 distribution
if z7_streaks['streaks']:
    print("\n  Streak Distribution (Z7):")
    unique_lengths = sorted(set(z7_streaks['streaks']))
    for length in unique_lengths[:10]:
        count = z7_streaks['streaks'].count(length)
        percentage = (count / z7_streaks['total_streaks']) * 100
        print(f"    {length} day(s): {count:>3} occurrences ({percentage:>5.1f}%)")
    if len(unique_lengths) > 10:
        print(f"    ... and {len(unique_lengths) - 10} more unique streak lengths")

print("\n" + "=" * 70)


  2.1. MAXIMUM DRAWDOWN ANALYSIS

Z6:
----------------------------------------------------------------------
  Maximum Drawdown.................................. -32.55%
  Peak Date......................................... 2018-03-08
  Trough Date....................................... 2018-03-25
  Recovery Date..................................... 2018-07-08
  Days to Trough.................................... 17
  Total Duration (days)............................. 122

Z7:
----------------------------------------------------------------------
  Maximum Drawdown.................................. -20.55%
  Peak Date......................................... 2011-10-30
  Trough Date....................................... 2011-11-02
  Recovery Date..................................... 2011-11-14
  Days to Trough.................................... 3
  Total Duration (days)............................. 15

  2.2. MAXIMUM DAILY LOSS ANALYSIS

Z6:
-------------------------------------------

Z6+ exhibits deep **drawdowns (max -32.6%)** driven primarily by multi-day loss clustering rather than isolated shocks, with a **worst day of -22.97%**. While the median losing streak is one day, 90% of loss sequences last no longer than three days, with rare tail events extending to seven consecutive days.

Z7 displays a smaller maximum **drawdown (-20.6%)**, with a **worst day of -15.32%**, and lower return volatility, but features heavier loss-clustering tails, with rare streaks extending up to ten consecutive trading days, despite a median losing streak of one day.